### We need to dowload pollutant data from the Ayuntamiento de Madrid and weather data from AEMET:

# AEMET data:

In [1]:
import numpy as np
import pandas as pd
import urllib
import json
import requests
from calendar import monthrange
from time import sleep

In [9]:
# get 'indicativo' of the station(s) of interest:
# need to create a file called 'api_key_aemet.txt' with only the API key given by AEMET
# then we read it:
stations_names = ['retiro','arganda'] #list of key words to look for
look_for = 'nombre' #'provincia' #search for this field of the json
querystring = {"api_key": open('api_key_aemet.txt','r').read()}
url = "https://opendata.aemet.es/opendata/api/valores/climatologicos/inventarioestaciones/todasestaciones"
response = requests.request( "GET", url= url, params = querystring).json()
meta_stations = requests.request('GET', url = response['metadatos'], params=querystring).json()
id_stations = requests.request('GET', url = response['datos'], params=querystring).json()
# # Store the JSON data in a file
# with open("inventario_todas_estaciones_AEMET.json", "w") as file:
#     json.dump(id_stations, file)
# json.dump( meta_stations, open('meta_stations_locations.json','w'))
[i for i in id_stations if any(j in i[look_for].lower() for j in stations_names)]

[{'latitud': '401842N',
  'provincia': 'MADRID',
  'altitud': '533',
  'indicativo': '3182Y',
  'nombre': 'ARGANDA DEL REY',
  'indsinop': '',
  'longitud': '032952W'},
 {'latitud': '402443N',
  'provincia': 'MADRID',
  'altitud': '667',
  'indicativo': '3195',
  'nombre': 'MADRID, RETIRO',
  'indsinop': '08222',
  'longitud': '034041W'}]

In [3]:
indicativo = 3195
#we make this query to initialize the columns of the data frame
#the url was picked from the aemet API in the section "valores-climatologicos"
retiro_query = requests.request('GET',
                                url=('https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/'
                                f'2010-01-01T00%3A00%3A00UTC/fechafin/2010-06-31T23%3A59%3A59UTC/estacion/{indicativo}')
                                , params=querystring)
print(retiro_query.text)
retiro_data = requests.request('GET', url=retiro_query.json()['datos'],
                               params=querystring)
#print(retiro_data.text)
initial_query = pd.json_normalize(retiro_data.json()) #now we have a dataframe
#display(initial_query)

{
  "descripcion" : "exito",
  "estado" : 200,
  "datos" : "https://opendata.aemet.es/opendata/sh/bd1cc43d",
  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"
}


In [4]:
# now we automatize this proccess, making querys for each 6 months
# since AEMET does not let us request data from one station in bigger periods:

station = 3195
metadata = []
in_year = 2010 #intial year
fin_year = 2023 #final year

## intialize empty dataset:
aemet_data = pd.DataFrame(columns=initial_query.columns)
n_year = fin_year - in_year + 1

for i in range(n_year):
    mes1 = '01'
    mes2 = '06'
    year = str(in_year + i)
    print('Iteration: ',i+1,', Year: ',year)
    #first 6 months, until 30 of June:
    
    req1 = ('https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/'
           f'{year}-{mes1}-01T00%3A00%3A00UTC/fechafin/{year}-{mes2}-30T23%3A59%3A59UTC/estacion/{station}')
                            
    first_half = requests.request(method='GET',
                           url=req1, params=querystring).json()
    
    #second half of the year, from 1 of July:
    mes3 = '07'
    mes4 = '12'
    req2 = ('https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/'
           f'{year}-{mes3}-01T00%3A00%3A00UTC/fechafin/{year}-{mes4}-31T23%3A59%3A59UTC/estacion/{station}')
    second_half = requests.request(method='GET',
                           url=req2, params=querystring).json()
    
    # wait for aemet in case we are making too many requests: ################
    while (first_half['descripcion'] != 'exito') or (second_half['descripcion'] != 'exito'):
        sleep(1)
        print('sleep...')
        first_half = requests.request(method='GET',
                           url=req1, params=querystring).json()
        second_half = requests.request(method='GET',
                           url=req2, params=querystring).json()
    ########################################################################
    url1 = first_half['datos']
    url2 = second_half['datos']
    
    # pick the data and convert to DataFrames through another request:
    # (i'm betting that if we have waited enough to make the two former requests,
    # these would be requested without the need to wait more)
    df1 = pd.json_normalize(
        requests.request(method='GET', url=url1, params=querystring).json() )
    df2 = pd.json_normalize(
        requests.request(method='GET', url=url2, params=querystring).json() )
    
    
    metadata.append(
        requests.request(method='GET', url=first_half['metadatos'], params=querystring).json() )
    metadata.append(
        requests.request(method='GET', url=second_half['metadatos'], params=querystring).json() )


    aemet_data = pd.concat([aemet_data, df1, df2])
#print(aemet_data.to_string()) #to check every line of the dataset
display(aemet_data)

Iteration:  1 , Year:  2010


Iteration:  2 , Year:  2011
Iteration:  3 , Year:  2012
Iteration:  4 , Year:  2013
Iteration:  5 , Year:  2014
Iteration:  6 , Year:  2015
Iteration:  7 , Year:  2016
Iteration:  8 , Year:  2017
Iteration:  9 , Year:  2018
Iteration:  10 , Year:  2019
Iteration:  11 , Year:  2020
Iteration:  12 , Year:  2021
Iteration:  13 , Year:  2022
Iteration:  14 , Year:  2023


,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,horaracha,presMax,horaPresMax,presMin,horaPresMin,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
0,2010-01-01,3195,"MADRID, RETIRO",MADRID,667,"5,6","0,1","3,6",Varias,"7,6",...,13:20,"938,5",24,"928,5",00,80,87,Varias,68,19:30
1,2010-01-02,3195,"MADRID, RETIRO",MADRID,667,"5,0","0,0","2,6",03:40,"7,5",...,01:00,"942,8",Varias,"938,5",00,84,95,Varias,74,00:00
2,2010-01-03,3195,"MADRID, RETIRO",MADRID,667,"5,8","6,7","4,2",02:30,"7,4",...,15:20,"941,6",03,"936,4",24,85,94,Varias,78,Varias
3,2010-01-04,3195,"MADRID, RETIRO",MADRID,667,"7,8","9,5","6,4",Varias,"9,2",...,06:20,"936,4",00,"924,2",24,96,98,Varias,92,23:59
4,2010-01-05,3195,"MADRID, RETIRO",MADRID,667,"8,0","0,0","6,2",23:59,"9,8",...,21:30,"926,0",24,"922,6",06,88,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2023-12-27,3195,"MADRID, RETIRO",MADRID,667,"4,6","0,0","-0,2",07:50,"9,3",...,14:00,"947,9",23,"945,1",14,76,99,Varias,55,14:30
180,2023-12-28,3195,"MADRID, RETIRO",MADRID,667,"4,0","0,0","0,9",04:00,"7,1",...,06:50,"948,9",11,"946,7",14,78,95,00:40,67,15:40
181,2023-12-29,3195,"MADRID, RETIRO",MADRID,667,"6,1","0,0","3,7",03:40,"8,5",...,13:30,"947,7",00,"943,5",15,79,89,Varias,54,15:00
182,2023-12-30,3195,"MADRID, RETIRO",MADRID,667,"6,2","0,0","1,8",05:50,"10,7",...,20:20,"944,8",11,"940,9",16,79,97,08:30,61,12:40


In [33]:
metadata_all_equal = len(metadata)-sum([i==metadata[0] for i in metadata]) #all components are the same!!
if metadata_all_equal == 0:
    print('all components are the same!!')
else:
    print('not all downloaded jsons are the same...')

all components are the same!!


In [6]:
# so the metadata is the same for every request, a rather visual check but not general as the one before:
from random import randint, seed
seed(1)
rand_check = randint(0, len(metadata)-1)
print('random element:', rand_check)
print('First element equal to random element?: ',metadata[0]==metadata[rand_check]) #the first element is equal to a random election
print(metadata[0])
print(metadata[rand_check])

random element: 4
First element equal to?:  True
{'unidad_generadora': 'Servicio del Banco Nacional de Datos Climatológicos', 'periodicidad': '1 vez al día, con un retardo de 4 días', 'descripcion': 'Climatologías diarias', 'formato': 'application/json', 'copyright': '© AEMET. Autorizado el uso de la información y su reproducción citando a AEMET como autora de la misma.', 'notaLegal': 'https://www.aemet.es/es/nota_legal', 'campos': [{'id': 'fecha', 'descripcion': 'fecha del dia (AAAA-MM-DD)', 'tipo_datos': 'string', 'requerido': True}, {'id': 'indicativo', 'descripcion': 'indicativo climatológico', 'tipo_datos': 'string', 'requerido': True}, {'id': 'nombre', 'descripcion': 'nombre (ubicación) de la estación', 'tipo_datos': 'string', 'requerido': True}, {'id': 'provincia', 'descripcion': 'provincia de la estación', 'tipo_datos': 'string', 'requerido': True}, {'id': 'altitud', 'descripcion': 'altitud de la estación en m sobre el nivel del mar', 'tipo_datos': 'float', 'unidad': 'm', 'requ

In [7]:
# # save the data as .csv:
# aemet_data.to_csv('AEMET_requested_data.csv',sep=';', index = False)
# # save metadata:

# # Store the JSON data in a file
# with open("metadata_aemet.json", "w") as file:
#     json.dump(metadata[0], file)

# Ayuntamiento de Madrid data:

In [8]:
last_year= 23
# the name of the dowloaded .csv files has been unified by hand to:
# "datos"+"last two digits of the year"+"csv extension"
# we needed to change by hand this file names
data_name=[f'datos{i}' for i in range(10,last_year+1)]
print(data_name)

['datos10', 'datos11', 'datos12', 'datos13', 'datos14', 'datos15', 'datos16', 'datos17', 'datos18', 'datos19', 'datos20', 'datos21', 'datos22', 'datos23']


In [9]:
est=[35, 4,8,49,56] # stations of interes: 35 plaza del carmen
df_est=[] #store datasets corresponding each station

#initialization out of the loop:
datos=pd.read_csv(data_name[0]+'.csv',sep=';')
df=datos.loc[datos['ESTACION']==est[0],:]
#df

## "transpose data" as desired:
#useful index for the first column we want to pick:
d1=datos.columns.get_loc('D01')

#initialiaze data matrix and final data frame for one station throught the years:
saved_variables = list(datos.columns[:d1]) #variables that don't need to be transposed
new_variables = ['DIA', 'D', 'V']
col_names = pd.Index(saved_variables + new_variables)
final_df=pd.DataFrame(columns=col_names)
p=len(col_names)

##start loop over the desired stations:
for d in est:
    print('Station ayto:',d)
    final_df=pd.DataFrame(columns=col_names)
    ##start loop over the years:
    for k in range(len(data_name)):
        print('Year: ',k+2010)
        datos=pd.read_csv(data_name[k]+'.csv',sep=';')
        df=datos.loc[datos['ESTACION']== d ,:]
        #df

        ## "transpose data" as desired:
        #useful index:
        d1=datos.columns.get_loc('D01')

        ## make the data matrix:
        n=df.shape[0] #number of original observations

        #initialize the matrix data
        new_data=np.zeros([n*31,p],dtype=object)
        # print(f'Number of new instances: {new_data.shape[0]}\nNumber of new total variables: {new_data.shape[1]}')
        
        # input our data in our initialized matrix:
        indexes=col_names.get_indexer(saved_variables)
        new_indexes=col_names.get_indexer(new_variables)
        for i in range(0,n):
            #repeated values
            new_data[(i*31):(i*31+31),indexes] = [df.iloc[i,indexes].values]*31
            #transposing data and validation:
            new_data[(i*31):(i*31+31),new_indexes] = np.column_stack([np.arange(1,31+1),
                                                                    df.iloc[i,d1::2].values,
                                                                    df.iloc[i,(d1+1)::2].values])
        new_df=pd.DataFrame(new_data, columns = col_names)
        final_df=pd.concat([final_df,new_df], ignore_index=True)
        new_df
        ## end loop over the years
        
    # convert columns into one data column
    final_df['FECHA'] = (final_df['ANO'].astype('str')+'-'+
                        final_df['MES'].astype('str').str.zfill(2)+'-'+
                        final_df['DIA'].astype('str').str.zfill(2))
    final_df = final_df.drop(['ANO','MES','DIA'], axis=1) #delete not useful columns
    final_df['FECHA'] = pd.to_datetime(final_df['FECHA'], errors='coerce')
    final_df = final_df.dropna() #drop fake dates
    df_est.append(final_df)
## end loop over the stations

Station ayto: 35
Year:  2010
Year:  2011
Year:  2012
Year:  2013
Year:  2014
Year:  2015
Year:  2016
Year:  2017
Year:  2018
Year:  2019
Year:  2020
Year:  2021
Year:  2022
Year:  2023
Station ayto: 4
Year:  2010
Year:  2011
Year:  2012
Year:  2013
Year:  2014
Year:  2015
Year:  2016
Year:  2017
Year:  2018
Year:  2019
Year:  2020
Year:  2021
Year:  2022
Year:  2023
Station ayto: 8
Year:  2010
Year:  2011
Year:  2012
Year:  2013
Year:  2014
Year:  2015
Year:  2016
Year:  2017
Year:  2018
Year:  2019
Year:  2020
Year:  2021
Year:  2022
Year:  2023
Station ayto: 49
Year:  2010
Year:  2011
Year:  2012
Year:  2013
Year:  2014
Year:  2015
Year:  2016
Year:  2017
Year:  2018
Year:  2019
Year:  2020
Year:  2021
Year:  2022
Year:  2023
Station ayto: 56
Year:  2010
Year:  2011
Year:  2012
Year:  2013
Year:  2014
Year:  2015
Year:  2016
Year:  2017
Year:  2018
Year:  2019
Year:  2020
Year:  2021
Year:  2022
Year:  2023


In [10]:
# # save data:
# for j in range(len(est)):
#     #display(df_est[j]) #visualize data
#     #save all data as .csv
#     df_est[j].to_csv('estacion'+f'{est[j]}'+'.csv',sep=';', index = False)

# MERGE THE DATA:

In [11]:
aemet_data = pd.read_csv(filepath_or_buffer='AEMET_requested_data.csv',sep=';')
aemet_data['fecha'] = pd.to_datetime(aemet_data['fecha']) #correct datatype
display(aemet_data)

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,horaracha,presMax,horaPresMax,presMin,horaPresMin,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
0,2010-01-01,3195,"MADRID, RETIRO",MADRID,667,"5,6","0,1","3,6",Varias,"7,6",...,13:20,"938,5",24,"928,5",00,80.0,87.0,Varias,68.0,19:30
1,2010-01-02,3195,"MADRID, RETIRO",MADRID,667,"5,0","0,0","2,6",03:40,"7,5",...,01:00,"942,8",Varias,"938,5",00,84.0,95.0,Varias,74.0,00:00
2,2010-01-03,3195,"MADRID, RETIRO",MADRID,667,"5,8","6,7","4,2",02:30,"7,4",...,15:20,"941,6",03,"936,4",24,85.0,94.0,Varias,78.0,Varias
3,2010-01-04,3195,"MADRID, RETIRO",MADRID,667,"7,8","9,5","6,4",Varias,"9,2",...,06:20,"936,4",00,"924,2",24,96.0,98.0,Varias,92.0,23:59
4,2010-01-05,3195,"MADRID, RETIRO",MADRID,667,"8,0","0,0","6,2",23:59,"9,8",...,21:30,"926,0",24,"922,6",06,88.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5107,2023-12-27,3195,"MADRID, RETIRO",MADRID,667,"4,6","0,0","-0,2",07:50,"9,3",...,14:00,"947,9",23,"945,1",14,76.0,99.0,Varias,55.0,14:30
5108,2023-12-28,3195,"MADRID, RETIRO",MADRID,667,"4,0","0,0","0,9",04:00,"7,1",...,06:50,"948,9",11,"946,7",14,78.0,95.0,00:40,67.0,15:40
5109,2023-12-29,3195,"MADRID, RETIRO",MADRID,667,"6,1","0,0","3,7",03:40,"8,5",...,13:30,"947,7",00,"943,5",15,79.0,89.0,Varias,54.0,15:00
5110,2023-12-30,3195,"MADRID, RETIRO",MADRID,667,"6,2","0,0","1,8",05:50,"10,7",...,20:20,"944,8",11,"940,9",16,79.0,97.0,08:30,61.0,12:40


In [12]:
# 14 years * 365 + 3 bisiesto year =
14*365+3 #this 5113 does not match, have a missing day from aemet 5112 rows

#lets create a range of dates using pandas:
range_dates=pd.date_range(start='2010-01-01', end='2023-12-31').values #as expected we have 5113 days
# see difference:
set(range_dates) - set(aemet_data['fecha'].values) #  it seems that aemet do not have data for the day 2022-03-14

{numpy.datetime64('2022-03-14T00:00:00.000000000')}

In [13]:
# station 35 is Plaza del Carmen,
# we will focus for the moment on NO2 (Magnitud 08):
ayto35 = pd.read_csv('estacion35.csv', sep=';')
# load dates correctly as dates:
ayto35['FECHA'] = pd.to_datetime(ayto35['FECHA'])
display(ayto35)
no2 = ayto35.iloc[ayto35['MAGNITUD'].values==8,:].reset_index()
display(no2)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,D,V,FECHA
0,28,79,35,1,28079035_1_38,10.0,V,2010-01-01
1,28,79,35,1,28079035_1_38,11.0,V,2010-01-02
2,28,79,35,1,28079035_1_38,14.0,V,2010-01-03
3,28,79,35,1,28079035_1_38,13.0,V,2010-01-04
4,28,79,35,1,28079035_1_38,13.0,V,2010-01-05
...,...,...,...,...,...,...,...,...
30307,28,79,35,14,28079035_14_6,6.0,V,2023-12-27
30308,28,79,35,14,28079035_14_6,2.0,V,2023-12-28
30309,28,79,35,14,28079035_14_6,1.0,V,2023-12-29
30310,28,79,35,14,28079035_14_6,6.0,V,2023-12-30


,index,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,D,V,FECHA
0,1095,28,79,35,8,28079035_8_8,24.0,V,2010-01-01
1,1096,28,79,35,8,28079035_8_8,46.0,V,2010-01-02
2,1097,28,79,35,8,28079035_8_8,59.0,V,2010-01-03
3,1098,28,79,35,8,28079035_8_8,63.0,V,2010-01-04
4,1099,28,79,35,8,28079035_8_8,54.0,V,2010-01-05
...,...,...,...,...,...,...,...,...,...
5047,29699,28,79,35,8,28079035_8_8,52.0,V,2023-12-27
5048,29700,28,79,35,8,28079035_8_8,55.0,V,2023-12-28
5049,29701,28,79,35,8,28079035_8_8,54.0,V,2023-12-29
5050,29702,28,79,35,8,28079035_8_8,46.0,V,2023-12-30


In [14]:
# there are missing dates, lets check how many have each year:
[no2.loc[no2['FECHA'].dt.year==i,:].shape[0] for i in range(2010,2023+1)]
# we have a problem in the last year: 2023,
# checking the ayto .csv we can check that there are missing 
# two months of data (October 31 and November 30), 61: 365-304=61
# we will work with the data that we have

[365, 365, 366, 365, 365, 365, 366, 365, 365, 365, 366, 365, 365, 304]

In [15]:
# double check with the set difference using the correct range of dates:
dif=set(range_dates) - set(no2['FECHA'].values)
print(len(dif))
display(sorted(dif))

61


[numpy.datetime64('2023-10-01T00:00:00.000000000'),
 numpy.datetime64('2023-10-02T00:00:00.000000000'),
 numpy.datetime64('2023-10-03T00:00:00.000000000'),
 numpy.datetime64('2023-10-04T00:00:00.000000000'),
 numpy.datetime64('2023-10-05T00:00:00.000000000'),
 numpy.datetime64('2023-10-06T00:00:00.000000000'),
 numpy.datetime64('2023-10-07T00:00:00.000000000'),
 numpy.datetime64('2023-10-08T00:00:00.000000000'),
 numpy.datetime64('2023-10-09T00:00:00.000000000'),
 numpy.datetime64('2023-10-10T00:00:00.000000000'),
 numpy.datetime64('2023-10-11T00:00:00.000000000'),
 numpy.datetime64('2023-10-12T00:00:00.000000000'),
 numpy.datetime64('2023-10-13T00:00:00.000000000'),
 numpy.datetime64('2023-10-14T00:00:00.000000000'),
 numpy.datetime64('2023-10-15T00:00:00.000000000'),
 numpy.datetime64('2023-10-16T00:00:00.000000000'),
 numpy.datetime64('2023-10-17T00:00:00.000000000'),
 numpy.datetime64('2023-10-18T00:00:00.000000000'),
 numpy.datetime64('2023-10-19T00:00:00.000000000'),
 numpy.datet

In [16]:
# adding the pollutant information to the waether information:
all_data = aemet_data.merge(no2, left_on='fecha', right_on='FECHA', how='inner') #'inner' chooses the intersection
all_data = all_data.drop('FECHA', axis=1)
display(all_data)
# # here i should check variables of interest and NA's
# all_data.to_csv('ALL_DATA.csv',sep=';', index = False) #save to .csv


,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,hrMin,horaHrMin,index,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,D,V
0,2010-01-01,3195,"MADRID, RETIRO",MADRID,667,"5,6","0,1","3,6",Varias,"7,6",...,68.0,19:30,1095,28,79,35,8,28079035_8_8,24.0,V
1,2010-01-02,3195,"MADRID, RETIRO",MADRID,667,"5,0","0,0","2,6",03:40,"7,5",...,74.0,00:00,1096,28,79,35,8,28079035_8_8,46.0,V
2,2010-01-03,3195,"MADRID, RETIRO",MADRID,667,"5,8","6,7","4,2",02:30,"7,4",...,78.0,Varias,1097,28,79,35,8,28079035_8_8,59.0,V
3,2010-01-04,3195,"MADRID, RETIRO",MADRID,667,"7,8","9,5","6,4",Varias,"9,2",...,92.0,23:59,1098,28,79,35,8,28079035_8_8,63.0,V
4,2010-01-05,3195,"MADRID, RETIRO",MADRID,667,"8,0","0,0","6,2",23:59,"9,8",...,NaN,NaN,1099,28,79,35,8,28079035_8_8,54.0,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5046,2023-12-27,3195,"MADRID, RETIRO",MADRID,667,"4,6","0,0","-0,2",07:50,"9,3",...,55.0,14:30,29699,28,79,35,8,28079035_8_8,52.0,V
5047,2023-12-28,3195,"MADRID, RETIRO",MADRID,667,"4,0","0,0","0,9",04:00,"7,1",...,67.0,15:40,29700,28,79,35,8,28079035_8_8,55.0,V
5048,2023-12-29,3195,"MADRID, RETIRO",MADRID,667,"6,1","0,0","3,7",03:40,"8,5",...,54.0,15:00,29701,28,79,35,8,28079035_8_8,54.0,V
5049,2023-12-30,3195,"MADRID, RETIRO",MADRID,667,"6,2","0,0","1,8",05:50,"10,7",...,61.0,12:40,29702,28,79,35,8,28079035_8_8,46.0,V


In [17]:
print("NA in each row: #of NA | frecuancy of that number of NA's")
print(all_data.isnull().sum(axis=1).value_counts())
print('\nNA in each column:')
print(all_data.isnull().sum(axis=0).to_string())


NA in each row: #of NA | frecuancy of that number of NA's
0     3986
4      942
1       36
3       31
5       16
18      11
9       10
8        8
7        5
16       2
2        1
11       1
10       1
6        1
Name: count, dtype: int64

NA in each column:
fecha               0
indicativo          0
nombre              0
provincia           0
altitud             0
tmed               14
prec               35
tmin               14
horatmin           14
tmax               14
horatmax           16
dir               653
velmedia          617
racha             653
horaracha         654
presMax            28
horaPresMax        28
presMin            27
horaPresMin        28
hrMedia            35
hrMax             398
horaHrMax         398
hrMin             399
horaHrMin         400
index               0
PROVINCIA           0
MUNICIPIO           0
ESTACION            0
MAGNITUD            0
PUNTO_MUESTREO      0
D                   0
V                   0


In [18]:
all_data.columns

Index(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec',
       'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha',
       'horaracha', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin',
       'hrMedia', 'hrMax', 'horaHrMax', 'hrMin', 'horaHrMin', 'index',
       'PROVINCIA', 'MUNICIPIO', 'ESTACION', 'MAGNITUD', 'PUNTO_MUESTREO', 'D',
       'V'],
      dtype='object')

# DO NOT RUN!!!!!!! 
Only keep in case of need, with the dates

AEMET:

In [19]:
# if we run this code very often or maybe there are many request to aemet we get an error an have to wait:
# for i in range(n_year*2):
#     print(i)
#     req1 = ('https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/'
#            f'{year}-{mes1}-01T00%3A00%3A00UTC/fechafin/{year}-{mes2}-31T23%3A59%3A59UTC/estacion/{station}')
                            
#     url1 = requests.request(method='GET',
#                            url=req1, params=querystring).json()
#     print(url1)
# 
# we need to try and wait a little, but ideally a will CODE
# A WHILE LOOP WITH A SLEEP TO WAIT AUTOMATICALLY!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [20]:
# # 14 years * 365 + 3 bisiesto year =
# 14*365+3 #this 5113 does not match, have a missing day from aemet 5112 rows

# #lets create a range of dates using pandas:
# range_dates=pd.date_range(start='2010-01-01', end='2023-12-31').values #as expected we have 5113 days
# # see difference:
# set(range_dates) - set(aemet_data['fecha'].values) #  it seems that aemet do not have data for the day 2022-03-14

In [21]:
# dates_counts = aemet_data['fecha'].value_counts()
# repeated_aemet_dates = dates_counts[ dates_counts > 1].index.sort_values()
# print(len(repeated_aemet_dates))
# print(repeated_aemet_dates) #all reapeated dates are from july first, is my problem requesting the aemet data 
# # it seems that i'm messing it up when requesting each year and splitting in half, repeating the last day
# # of the first request and the first day of the second request

In [22]:
# from time import sleep
# for i in range(50):
#     xd = (requests.request(method='GET',
#                            url=req1, params=querystring).json())
#     while xd['descripcion'] != 'exito':
#         print('sleeping')
#         sleep(1)
#         xd = (requests.request(method='GET',
#                            url=req1, params=querystring).json())
#         print(xd)
#     print(xd)

AYTO OF MADRID:

In [23]:
# # convert columns into one data column
# no2['FECHA'] = no2['ANO'].astype('str')+'-'+no2['MES'].astype('str').str.zfill(2)+'-'+no2['DIA'].astype('str').str.zfill(2)
# no2 = no2.drop(['ANO','MES','DIA'], axis=1)
# no2

In [24]:
# no2['FECHA']

In [25]:
# pd.to_datetime(no2['FECHA'], errors='coerce').dropna()

In [26]:
# # we have to delete many fake dates, as 31 of february, for example
# # and later see what happened with the AEMET data
# from calendar import monthrange
# #monthrange(2010, 1) #1st element which weekday is the first of the month, 2nd element number of days
# max_day = [monthrange(year=i[0], month=i[1])[1] for i in zip(no2['ANO'],no2['MES'])]
# no2.loc[no2['DIA'] > max_day, 'DIA'] = np.nan

In [27]:
# no2.shape
# no2.dropna()

In [28]:
# 14*12*31

In [29]:
# there are missing dates, since we should have make a fake arpox:
# 14*12*31=5208, lets see if each year has 12*31=372 days
[no2.loc[no2['FECHA'].dt.year==i,:].shape[0] for i in range(2010,2023+1)]
# we have a problem in the last year: 2023
# we have missing two months of data: 365-304=61
# there is missing info in the ayto madrid database,
# we will work with the data that we have

[365, 365, 366, 365, 365, 365, 366, 365, 365, 365, 366, 365, 365, 304]

MERGING FINAL DATES:

In [30]:
# # now we would mix only the data that we can: every data except 2023 October & November (lack of ayto) and the day 2022-03-14 (lack of AEMET)
# data_dates = set(aemet_data['fecha'].values) & set(no2['FECHA'].values)
# data_dates = sorted(data_dates) #sorting just in case (now we have a list not a set)
# print([i in data_dates for i in aemet_data['fecha'].values]) #2m22.5s too long, there has to be a better way: look pandas functions